University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2022  
Luigi Acerbi  

# Week 6 exercises

## 1. Leapfrog integration in HMC (4 points)

In this exercise we will experiment with the leapfrog integrator used in HMC. As a target, we will use the 2D correlated normal 
$$ \pi(\boldsymbol{\theta}) = \mathcal{N}(\boldsymbol{\theta} ;\; 0, \boldsymbol{\Sigma}_2), $$
where $ \boldsymbol{\Sigma}_2 = \begin{pmatrix} 1 & \rho \\ \rho & 1 \end{pmatrix}$ with $\rho = 0.998$.

1. Simulate the system for $L=10$ leapfrog steps starting from the origin $\boldsymbol{\theta}_0 = (0, 0)$ with momentum $r = (1, 1/3)$ with several values of $\epsilon$ and plot the resulting Metropolis-Hastings acceptance probabilities $a$ at the end of the leapfrog trajectory. Considering values of $\epsilon \in [0.001, 0.1]$:
  - What is the smallest $\epsilon$ that will yield acceptance probability below 60%? 
  - What is the largest $\epsilon$ that will yield an acceptance probability above 10%?
  - For this exercise, the required tolerance on the answer is 0.001.
2. Simulate the Hamiltonian system for 500 leapfrog steps with $\epsilon=0.05$ starting from the origin $\boldsymbol{\theta}_0 = (0, 0)$ with momentum $r = (1, 1/3)$. Plot the trajectory and the Euclidean distance from the starting point at each point. 
  - How many leapfrog steps do you need to take until the trajectory reaches the first local maximum distance from the origin?
  - How many leapfrog steps (since the start of the trajectory) will lead you to the first local minimum?

*Note*: This exercise tests only the deterministic leapfrog integration used within the HMC algorithm to produce a single proposal. **This procedure is completely deterministic.** You will not need any random number generators in your solution. In particular, you will not need to run the full HMC algorithm or make any actual accept/reject decisions.

## 2. Efficiency of "random walk" MCMC and HMC (4 points)

In this exercise we will test the efficiency of Metropolis-Hastings (also known as "random walk" MCMC) and HMC in terms of the distance covered in various situations. 

All the experiments are done using the same correlated normal target as in Exercise 1.
For Metropolis-Hastings MCMC, we will use as proposal $q(\boldsymbol{\theta}'; \boldsymbol{\theta}) = \mathcal{N}(\boldsymbol{\theta}'; \boldsymbol{\theta}, \sigma^2 \mathbf{I}_2)$ with $\sigma = 0.01$. For HMC, set $\epsilon=0.01$ and $L$ as specified below.

Distance covered here means the distance between the final point and the starting point of the chain, ignoring any intermediate steps.

1. Always starting at $\boldsymbol{\theta}_0 = (0, 0)$, run Metropolis-Hastings MCMC 100 times for 1 step and 100 times for 100 steps. Compute the average Euclidean distance covered from the starting point in each case. Compare the 100 step distance with the theoretical random walk scaling of $\sqrt{n}$ over $n$ steps.
2. Always starting at $\boldsymbol{\theta}_0 = (0, 0)$, run HMC 100 times for 1 step with $L=1$ and 100 times for 1 step with $L=100$. Compute the average Euclidean distance covered from the starting point in each case. Compare the 100 step distance with the theoretical random walk scaling of $\sqrt{n}$ over $n$ steps.
3. Repeat task 1 starting at $\boldsymbol{\theta}_0 = (-2, 2)$.
4. Repeat task 2 starting at $\boldsymbol{\theta}_0 = (-2, 2)$.
5. In each case, report the two average Euclidean distances in Moodle. In which cases do the algorithms move more quickly than would be expected for a random walk?

## 3. NUTS simulation (6 points)

In this exercise we will experiment with the build-tree operation of the NUTS algorithm. We will use a simplified version of the algorithm with the following differences from the original:
* We will only extend the path forward instead of choosing the direction randomly.
* We will check for the termination condition only from the entire newly added path segment, not all subpaths as NUTS really would.

Your tasks:
1. Implement a simplified version of NUTS that works as follows:
  - Start at iteration $j = 1$.
  - In each iteration $j$, with $j = 1,2,\ldots$, set $L = 2^{j-1}$ and run $L$ leapfrog steps forward. Deterministically move to the new position (i.e., there is no acceptance check). 
  - At the end of each iteration, a "termination check" is run. The algorithm stops when the first newly simulated position $(\boldsymbol{\theta}^-, \mathbf{r}^-)$ and the last newly simulated position $(\boldsymbol{\theta}^+, \mathbf{r}^+)$ of the trajectory simulated in the current iteration segment satisfy
$$ (\boldsymbol{\theta}^+ - \boldsymbol{\theta}^-) \cdot \mathbf{r}^- < 0 \quad \text{or} \quad (\boldsymbol{\theta}^+ - \boldsymbol{\theta}^-) \cdot \mathbf{r}^+ < 0. $$
2. Run your algorithm on the 2D correlated normal with $\rho = 0.998$, starting from the origin $\boldsymbol{\theta}_0 = (0, 0)$ with momentum $\mathbf{r} = (1, 1/3)$ with $\epsilon=0.05$. At which iteration $j$ of the algorithm will the termination criterion be triggered for the first time? Report the iteration number in Moodle.
3. Compute the distance from the origin of the point at which the termination criterion is triggered, and report it in Moodle.
4. NUTS includes a special condition to simulate $u \sim \mathrm{Uniform}(0, \exp(-H(\boldsymbol{\theta}_0, \mathbf{r}_0))$ and ignoring all points where $u > \exp(-H(\boldsymbol{\theta}, \mathbf{r}))$. Assuming $u = 0.98\exp(-H(\boldsymbol{\theta}_0, \mathbf{r}_0))$ (a pretty extreme value), how many points from the trajectory of $2^j$ points generated during the first $j$ iterations (including the initial state and the one during which the termination condition was triggered first) would be included in the next step of the algorithm when choosing the next point?

## 4. HMC sampling for posterior inference (6 points)

In this task we will apply HMC sampling for posterior inference of a linear regression model.
We will use data from the Framingham Heart Study that studies the association between heart disease and its causes.
A description of the data and its fields can be found at http://www.maths.utas.edu.au/DHStat/Data/Flow.html (see also problem 10.4 in the computer tasks done in class).

We will use a linear regression model
$$ y_i = \mathbf{x}_i \boldsymbol{\beta} + \epsilon_i, $$
where $\mathbf{x}_i$ is a row vector of input variables and $\boldsymbol{\beta}$ is a column vector of regression coefficients, and the noise term $\epsilon_i$ follows the normal $\mathcal{N}(0, \sigma_y^2)$. We will not need a constant intercept term in the model because we will standardise the data.

Assuming the residuals $\epsilon_i, i=1,\dots,n$ are independent, the log-likelihood of the model is
$$ \log p(y | x, \boldsymbol{\beta}, \sigma_y) = \sum_{i=1}^n \log p(y_i | \boldsymbol{\beta}, x_i, \sigma_y) = \sum_{i=1}^n \log \mathcal{N}(y_i; \; \mathbf{x}_i \boldsymbol{\beta}, \sigma_y^2). $$

1. Load the data using the below code. Standardise the data (both $x$ and $y$) by subtracting the mean and dividing each variable by **two** standard deviations (see http://www.stat.columbia.edu/~gelman/research/published/standardizing7.pdf). Fit all the models using the standardised data as this makes sampling a lot easier.
2. Using the above log-likelihood and priors
$$ p(\boldsymbol{\beta}) = \prod_j p(\beta_j) = \prod_j \mathcal{N}(\beta_j;\; 0, 2^2), \\ p(\sigma_y) = \mathrm{Gamma}(\sigma_y;\; k=10, \theta=0.1),$$
use HMC or NUTS to draw samples from the posterior distribution $p(\boldsymbol{\beta}, \sigma_y | x, y)$, when the prediction target variable $y$ is 'SBP' and the input variables $x$ are 'FRW', 'AGE' and 'CHOL'.

To ensure the standard deviation $\sigma_y$ is positive, express it as $\sigma_y = \exp(s_y)$, where $s_y$ is unbounded. Remember to apply the density transformation to transform the prior over $\sigma_y$ to that over $s_y$!

Make sure your sampler is properly tuned so that results will be reliable.

Report your estimates of the posterior means and standard deviations of the coefficient in $\beta$ for 'FRW', and of $s_y$. (*Note*: This should be $s_y$, not $\sigma_y$!)

Implementations of the gamma log-pdf needed for the prior that are compatible with Autograd and PyTorch are provided below for your convenience. (Autograd does not support `scipy.stats.gamma.logpdf` to the level we need. The PyTorch library implementation uses a different parametrisation, so the below implementation is easier for consistency.)

The required tolerance is $\pm 0.01$.

*Note*: The NUTS implementation provided in the course materials sometimes gets stuck and produces a sample with really small variance. If you encounter such behaviour, you should disregard such results.

In [ ]:
# For autograd
import pandas as pd
import autograd.numpy as np
import autograd.scipy.special as scs
import autograd

def standardize(x):
    raise(NotImplementedError("Not implemented yet"))

# load the data from CSV file using pandas
fram = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/fram.txt', sep='\t')
# convert the variables of interest to numpy arrays for autograd compatibility
# input: Framingham relative weight - the ratio of the subjects weight to the median weight for their sex-height group,
#   age and cholesterole level
x = fram[['FRW', 'AGE', 'CHOL']].values.astype(float)
# target: Systolic blood pressure, examination 1
y = fram['SBP'].values.squeeze().astype(float)

xs = standardize(x)
ys = standardize(y)

def gamma_logpdf(x, k, theta):
    """Gamma distribution log pdf."""
    return -scs.gammaln(k) - k*np.log(theta) + (k-1)*np.log(x) - x / theta

In [ ]:
# For PyTorch
import pandas as pd
import torch

def standardize(x):
    raise(NotImplementedError("Not implemented yet"))

# load the data from CSV file using pandas
fram = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/fram.txt', sep='\t')
# convert the variables of interest to numpy arrays for autograd compatibility
# input: Framingham relative weight - the ratio of the subjects weight to the median weight for their sex-height group,
#   age and cholesterole level
x = fram[['FRW', 'AGE', 'CHOL']].values.astype(float)
# target: Systolic blood pressure, examination 1
y = fram['SBP'].values.squeeze().astype(float)

x = torch.tensor(x, dtype=torch.double)
y = torch.tensor(y, dtype=torch.double)

xs = standardize(x)
ys = standardize(y)

def gamma_logpdf(x, k, theta):
    """Gamma distribution log pdf (PyTorch)."""    
    return -torch.lgamma(k) - k*torch.log(theta) + (k-1)*torch.log(x) - x / theta